# PX915 Individual Project Reproducible Result - Ben Gosling

#### Import Modules for Plasma Parameter Calculation and GP regression

In [1]:
# import epoch calculators
from Reflectivity_scripts.plasma_calc import * # stores class that calculates basic plasma paramters 
from Reflectivity_scripts.fields_calc import * # stores class that calculates field pramteres such as refelctivity 
from Reflectivity_scripts.run_epoch import * # stores functions which aid in the running of epoch simulations
from Reflectivity_scripts.sim_setup import * # stores functions which aid in the creation of setting up epoch runs

# Gaussian process regression scripts
from GP_scripts.gp import * # stores functions for performing GP regression for 1D input space

EPOCH requires you to specify an output directory which stores the input file to set up the simulation and store the output files. The python function below is used to create a directory within epoch_surra and populate it with one of the example input decks in the input_decks directory. 

In [2]:
# set name of the output directory
dir = 'Data_epoch'
sub_dirs = [f'Data_{i}' for i in range(1, 11)]
# input file/setup used throughout the report
input_file = 'example_input.deck'
# set initial laser intensity in W/cm^2 (varies between 1e14 - 1e16 in the report)
intensity = 1e16 # set initial laser intensity in W/cm^2 
# set density scale length in m (varies between 300e-6 - 100e-6 in the report) 
dens_scale_len = 500e-6 
# set the number of particles per cell (set to 2048 in the report)
# set to 100 to save time
ppc = 100


In [3]:
for i in range(len(sub_dirs)):
    epoch_sim_sub_dir(dir = dir, sub_dir= sub_dirs[i], input_file = input_file, I = intensity, Ln = dens_scale_len, ppc = ppc)

created directory Data_epoch/Data_1 and input.deck
Data_epoch directory already exists
created directory Data_epoch/Data_2 and input.deck
Data_epoch directory already exists
created directory Data_epoch/Data_3 and input.deck
Data_epoch directory already exists
created directory Data_epoch/Data_4 and input.deck
Data_epoch directory already exists
created directory Data_epoch/Data_5 and input.deck
Data_epoch directory already exists
created directory Data_epoch/Data_6 and input.deck
Data_epoch directory already exists
created directory Data_epoch/Data_7 and input.deck
Data_epoch directory already exists
created directory Data_epoch/Data_8 and input.deck
Data_epoch directory already exists
created directory Data_epoch/Data_9 and input.deck
Data_epoch directory already exists
created directory Data_epoch/Data_10 and input.deck
